In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')

In [3]:
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1771,tt0374563,17000000,2626800,Captivity,Elisha Cuthbert|Daniel Gillies|Pruitt Taylor V...,Roland JoffÃ©,When You Think The Worst Has Happened... Think...,"The sought-after images of top model, Jennifer...",96,Crime|Horror|Thriller,Foresight Unlimited|RAMCO|Captivity Productions,6/22/2007,4.5,2007
693,tt1000774,65000000,415252786,Sex and the City,Sarah Jessica Parker|Kim Cattrall|Cynthia Nixo...,Michael Patrick King,Get carried away.,A New York writer on sex and love is finally g...,145,Comedy|Drama|Romance,New Line Cinema|Darren Star Productions|Home B...,5/12/2008,6.1,2008
1670,tt0373889,150000000,938212738,Harry Potter and the Order of the Phoenix,Daniel Radcliffe|Rupert Grint|Emma Watson|Mich...,David Yates,Evil Must Be Confronted.,Returning for his fifth year of study at Hogwa...,138,Adventure|Fantasy|Family|Mystery,Cool Music|Warner Bros.|Heyday Films|Harry Pot...,6/28/2007,7.2,2007


In [4]:
data.head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [6]:
answers = {} # создадим словарь для ответов

split_values_vert = lambda value: value.split('|')


month_to_season = lambda month: "Spring" if 3 <= month <= 5 \
                      else "Summer" if 6 <= month <= 8 \
                      else "Autumn" if 9 <= month <= 11 \
                      else "Winter" if month == 12 or month == 1 or month == 2 \
                      else "error"


data.release_date = pd.to_datetime(data.release_date)
data.cast = data.cast.apply(split_values_vert)
data.genres = data.genres.apply(split_values_vert)
data.production_companies = data.production_companies.apply(split_values_vert)

data["profit"] = data.revenue - data.budget
data["season"] = data.apply(lambda row: month_to_season(int(row["release_date"].strftime("%m"))), axis=1)

def movie_info_by_index(data, index):
    """Возвращает строку с индексом фильма в таблице data, его название и IMDb идентификатор.
    Эта функция используется для помещение ответов на вопросы в словарь answers
    в строковой форме.
    
    Args:
        data: таблица с фильмами
        index: индекс записи фильма в таблице
    Returns:
        Возвращает строку с индексом фильма в таблице data, его название и IMDb идентификатор
        в формате 'index. original_title (imdb_id)'
    """
    movie_entry = data.loc[index]
    movie_title = movie_entry.original_title
    movie_imdb_id = movie_entry.imdb_id
    return f"{index}. {movie_title} ({movie_imdb_id})"

def create_dataframe_split_column_list(data, data_column, index_column):
    """Разделяет значения в заданной колонке исходной таблицы, где
    они представлены списком и формирует новую таблицу, в которой
    есть только колонка индекса с дублирующимися значениями и
    колонка с одиночными значениями вместо списков.
    
    Args:
        data: исходная таблица с данными
        data_column: название колонки, где в ячейках есть списки значений
        index_column: колонка данных, которая должна играть роль индексной
    Returns:
        Новая таблицу, в которой есть только колонка индекса с дублирующимися значениями и
        колонка с одиночными значениями вместо списков.
    """
    data_columns = data.columns.tolist()
    new_data = pd.DataFrame(data[data_column].tolist(), index=data[index_column]).stack()
    new_data = new_data.reset_index([0, index_column])
    new_data.columns = [index_column, data_column]
    return new_data

def create_dataframe_column_full(data, target_column_list):
    """Создаёт из исходной таблицы данных новую таблицу данных,
    в которой заданная колонка со списками значений разделяется
    с образованием нескольких строк таблицы, число которых равно
    числу элементов списка в каждой ячейке такой колонки.
    Результат аналогичен DataFrame.explode

    Args:
        data: исходная таблица с данными
        target_column_list: название колонки в таблице data,
                            где в ячейках находятся списки значений
    Returns:
        Новую таблицу данных, в которой заданная колонка со списками
        значений разделяется с образованием нескольких строк таблицы,
        число которых равно числу элементов списка в каждой ячейке такой колонки.
    """
    rows = list()

    def make_new_row(row):
        for cell_value in row[target_column_list]:
            new_row = row.to_dict()
            new_row[target_column_list] = cell_value
            rows.append(new_row)

    data.apply(make_new_row, axis=1)
    new_data = pd.DataFrame(rows)
    return new_data

In [7]:
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,season
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",2015-06-09,6.5,2015,1363528810,Summer
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",2015-05-13,7.1,2015,228436354,Spring
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",2015-03-18,6.3,2015,185238201,Spring
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",2015-12-15,7.5,2015,1868178225,Winter
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",2015-04-01,7.3,2015,1316249360,Spring


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [9]:
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # +

In [10]:
df_movie_max_budget = data.loc[data.budget == data.budget.max()]

answer_01 = movie_info_by_index(df_movie_max_budget, df_movie_max_budget.index[0])

print(f"Answer 01: '{answer_01}'")

Answer 01: '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'


**ВАРИАНТ 2**

In [11]:
answer_01 = movie_info_by_index(data, data.budget.idxmax())

print(f"Answer 01: '{answer_01}'")

Answer 01: '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'


# 2. Какой из фильмов самый длительный (в минутах)?

In [12]:
answers['2'] = '1157. Gods and Generals (tt0279111)' # +

In [13]:
df_movie_max_runtime = data.loc[data.runtime == data.runtime.max()]

answer_02 = movie_info_by_index(df_movie_max_runtime, df_movie_max_runtime.index[0])

print(f"Answer 02: '{answer_02}'")

Answer 02: '1157. Gods and Generals (tt0279111)'


**ВАРИАНТ 2**

In [14]:
answer_02 = movie_info_by_index(data, data.runtime.idxmax())

print(f"Answer 02: '{answer_02}'")

Answer 02: '1157. Gods and Generals (tt0279111)'


# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
answers['3'] = '768. Winnie the Pooh (tt1449283)' # +

In [16]:
answer_03 = movie_info_by_index(data, data.runtime.idxmin())

print(f"Answer 03: '{answer_03}'")

Answer 03: '768. Winnie the Pooh (tt1449283)'


# 4. Какова средняя длительность фильмов?


In [17]:
answers['4'] = '110' # +

In [18]:
answer_04 = round(data.runtime.mean())
print(f"Answer 04: '{answer_04}'")

Answer 04: '110'


# 5. Каково медианное значение длительности фильмов? 

In [19]:
answers['5'] = '107' # +

In [20]:
answer_05 = round(data.runtime.median())
print(f"Answer 05: '{answer_05}'")

Answer 05: '107'


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [21]:
answers['6'] = '239. Avatar (tt0499549)' # +

In [22]:
movie_index = data.profit.idxmax()
answer_06 = movie_info_by_index(data, movie_index)

print(f"Answer 06: '{answer_06}'")

Answer 06: '239. Avatar (tt0499549)'


# 7. Какой фильм самый убыточный? 

In [23]:
answers['7'] = '1245. The Lone Ranger (tt1210819)' # +

In [24]:
movie_index = data.profit.idxmin()
answer_07 = movie_info_by_index(data, movie_index)

print(f"Answer 07: '{answer_07}'")

Answer 07: '1245. The Lone Ranger (tt1210819)'


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
answers['8'] = '1478' # +

In [26]:
answer_08 = len(data.loc[data.revenue > data.budget])
print(f"Answer 08: '{answer_08}'")

Answer 08: '1478'


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [27]:
answers['9'] = '599. The Dark Knight (tt0468569)' # +

In [28]:
movies_2008 = data.loc[data.release_year == 2008]

movie_index = movies_2008.profit.idxmax()
answer_09 = movie_info_by_index(movies_2008, movie_index)

print(f"Answer 09: '{answer_09}'")

Answer 09: '599. The Dark Knight (tt0468569)'


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [29]:
answers['10'] = '1245. The Lone Ranger (tt1210819)' # +

In [30]:
movies_2012_2014 = data.loc[(2012 <= data.release_year) & (data.release_year <= 2014)]

movie_index = movies_2012_2014.profit.idxmin()
answer_10 = movie_info_by_index(movies_2012_2014, movie_index)

print(f"Answer 10: '{answer_10}'")

Answer 10: '1245. The Lone Ranger (tt1210819)'


# 11. Какого жанра фильмов больше всего?

In [31]:
answers['11'] = 'Drama' # +

In [32]:
genre_counter = Counter()
for index, row in data.iterrows():
    for genre in row["genres"]:
        genre_counter.update({genre: 1})
answer_11 = genre_counter.most_common(1)[0][0]

print(f"Answer 11: '{answer_11}'")

Answer 11: 'Drama'


ВАРИАНТ 2

In [33]:
data_by_genres = create_dataframe_split_column_list(data, "genres", "imdb_id")

genre_counter = Counter()
for index, row in data_by_genres.iterrows():
    genre_counter.update({row["genres"]: 1})

answer_11 = genre_counter.most_common(1)[0][0]
print(f"Answer 11: '{answer_11}'")

Answer 11: 'Drama'


ВАРИАНТ 3

In [34]:
new_data = create_dataframe_column_full(data, "genres")

genre_counter = Counter()
for index, row in new_data.iterrows():
    genre_counter.update({row["genres"]: 1})

answer_11 = genre_counter.most_common(1)[0][0]
print(f"Answer 11: '{answer_11}'")

Answer 11: 'Drama'


ВАРИАНТ 4

In [35]:
new_data = data.explode("genres")

genre_counter = Counter()
for index, row in new_data.iterrows():
    genre_counter.update({row["genres"]: 1})

answer_11 = genre_counter.most_common(1)[0][0]
print(f"Answer 11: '{answer_11}'")

Answer 11: 'Drama'


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [36]:
answers['12'] = 'Drama' # +

In [37]:
new_data = data.explode("genres")

genre_profitable = Counter()
for index, row in new_data.iterrows():
    if(row["profit"] > 0):
        genre_profitable.update({row["genres"]: 1})

answer_12 = genre_profitable.most_common(1)[0][0]
print(f"Answer 12: '{answer_12}'")

Answer 12: 'Drama'


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [38]:
answers['13'] = 'Peter Jackson' # +

In [39]:
director_revenues = dict()

for index, row in data.iterrows():
    director = row["director"]
    revenue = row["revenue"]
    director_revenues[director] = (director_revenues[director] if director in director_revenues else 0) + revenue

answer_13 = max(director_revenues, key=director_revenues.get)
print(f"Answer 13: '{answer_13}'")

Answer 13: 'Peter Jackson'


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [40]:
# Это один из вариантов ответа в тесте
# реально подсчёт выдаёт другой ответ по двум режиссёрам с
# максимальным числом фильмов в стиле Action,
# которых вообще нет в ответах теста: Michael Bay - 7, Paul W.S. Anderson - 7.
# Ручной просмотр моего скачанного с skillfactory CSV файла показал, что действитльно
# режиссёры Michael Bay - 7, Paul W.S. Anderson - 7,
# а режиссёры Robert Rodriguez - 6 и Ridley Scott - 6.
# Тем не менее в тесте есть два варианта ответа - не Ridley Scott и Robert Rodriguez.
# При этом верный вариант - Robert Rodriguez
answers['14'] = 'Robert Rodriguez' # +

In [41]:
data_genres = data.explode("genres")
answer_14 = data_genres[data_genres["genres"] == "Action"]["director"].value_counts(sort=True).head(10)
print(f"Answer 14:\n'{answer_14}'")

Answer 14:
'Michael Bay           7
Paul W.S. Anderson    7
Robert Rodriguez      6
Ridley Scott          6
Antoine Fuqua         6
Zack Snyder           5
Rob Cohen             5
Andrzej Bartkowiak    5
Brett Ratner          5
Roland Emmerich       5
Name: director, dtype: int64'


ВАРИАНТ 2

In [42]:
director_genres = Counter()
for index, row in data.iterrows():
    if "Action" in row["genres"]:
        director_genres.update({row["director"]: 1})

answer_14 = director_genres.most_common(10)
print(f"Answer 14:\n'{answer_14}'")

Answer 14:
'[('Michael Bay', 7), ('Paul W.S. Anderson', 7), ('Antoine Fuqua', 6), ('Ridley Scott', 6), ('Robert Rodriguez', 6), ('Brett Ratner', 5), ('Zack Snyder', 5), ('Roland Emmerich', 5), ('Andrzej Bartkowiak', 5), ('Tony Scott', 5)]'


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [43]:
answers['15'] = 'Chris Hemsworth' # +

In [44]:
data_cast = data[data["release_year"] == 2012].explode("cast")
pivot = pd.pivot_table(data_cast, index=["release_year"], columns=["cast"], values=["revenue"], aggfunc=np.sum)
actor_movie_revenues = pivot.iloc[0].sort_values(ascending=False)

answer_15 = actor_movie_revenues.index[0][1]
print(f"Answer 15: '{answer_15}' ({actor_movie_revenues.iloc[0]})")

Answer 15: 'Chris Hemsworth' (2027450773)


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [45]:
answers['16'] = 'Matt Damon' # +

In [46]:
categorie_A_budget_min = data["budget"].quantile(q=0.5) # a high budget movie, median()
actors_high_budget = data[data["budget"] >= categorie_A_budget_min].explode("cast")["cast"]\
                        .value_counts(sort=True, ascending=False)
answer_16 = actors_high_budget.index[0]
print(f"Answer 16: '{answer_16}' ({actors_high_budget.iloc[0]})")

Answer 16: 'Matt Damon' (23)


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [47]:
answers['17'] = 'Action' # +

In [48]:
data_cast = data.explode("cast")
data_Nicolas_Cage = data_cast[data_cast["cast"] == "Nicolas Cage"]
data_Nicolas_Cage_genres = data_Nicolas_Cage.explode("genres")
genres_counts = data_Nicolas_Cage_genres["genres"].value_counts(sort=True, ascending=False)

answer_17 = genres_counts.index[0]
print(f"Answer 17: '{answer_17}' ({genres_counts.iloc[0]})")

Answer 17: 'Action' (17)


# 18. Самый убыточный фильм от Paramount Pictures

In [49]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)' # +

In [50]:
data_companies = data.explode("production_companies")
data_Paramount = data_companies[data_companies["production_companies"] == "Paramount Pictures"]
most_unprofitable_film = data_Paramount[data_Paramount["profit"] == data_Paramount["profit"].min()].iloc[0]

answer_18 = f"{most_unprofitable_film['original_title']} ({most_unprofitable_film['imdb_id']})"
print(f"Answer 18: '{answer_18}'")

Answer 18: 'K-19: The Widowmaker (tt0267626)'


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [51]:
answers['19'] = '2015' # +

In [52]:
year_revenue = data.groupby(["release_year"])["revenue"].sum().sort_values(ascending=False)

answer_19 = year_revenue.index[0]
print(f"Answer 19: '{answer_19}' ({year_revenue.iloc[0]})")

Answer 19: '2015' (25449202382)


# 20. Какой самый прибыльный год для студии Warner Bros?

In [53]:
answers['20'] = '2014' # +

In [54]:
data_companies = data.explode("production_companies")
data_Warner = data_companies[data_companies["production_companies"].str.contains("Warner Bros")]
year_revenue_Warner = data_Warner.groupby("release_year")["profit"].sum().sort_values(ascending=False)

answer_20 = year_revenue_Warner.index[0]
print(f"Answer 20: '{answer_20}' ({year_revenue_Warner.iloc[0]})")

Answer 20: '2014' (2292949646)


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [55]:
answers['21'] = 'Сентябрь' # +, September

In [56]:
films_monthly = data.groupby(data["release_date"].dt.strftime("%B"))["imdb_id"] \
                    .count().sort_values(ascending=False)
films_monthly = films_monthly.rename("Movies by Month")
films_monthly.index = films_monthly.index.rename("month")

answer_21 = films_monthly.index[0]
print(f"Answer 21: '{answer_21}' ({films_monthly.iloc[0]})")

Answer 21: 'September' (227)


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [57]:
answers['22'] = '450' # +

In [58]:
films_monthly = data.groupby(data["release_date"].dt.strftime("%B"))["imdb_id"] \
                    .count().sort_values(ascending=False)
films_monthly = films_monthly.rename("Movies by Month")
films_monthly.index = films_monthly.index.rename("month")

answer_22 = films_monthly[["July", "June", "August"]].sum()
print(f"Answer 22: '{answer_22}'")

Answer 22: '450'


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [59]:
answers['23'] = 'Peter Jackson' # +

In [60]:
director_season = pd.pivot_table(data, index=["director"], columns=["season"], values=["imdb_id"],
                                 aggfunc="count", fill_value=0)

director_season_imdb_id = director_season["imdb_id"]
# director_season[director_season["Winter"] = ]
winter_director = director_season_imdb_id[director_season_imdb_id["Winter"] == director_season_imdb_id["Winter"].max()].index[0]

answer_23 = winter_director
print(f"Answer 23: '{answer_23}' ({director_season_imdb_id.loc[answer_23].Winter})")

Answer 23: 'Peter Jackson' (7)


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [61]:
answers['24'] = 'Four By Two Productions' # +

In [62]:
company_titles = data.explode("production_companies") \
                    .groupby("production_companies")["original_title"] \
                    .apply(lambda titles: np.mean(titles.str.len())) \
                    .sort_values(ascending=False)

answer_24 = company_titles.index[0]
print(f"Answer 24: '{answer_24}' ({company_titles.iloc[0]})")

Answer 24: 'Four By Two Productions' (83.0)


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [63]:
answers['25'] = 'Midnight Picture Show' # +

In [64]:
company_overview = data.explode("production_companies") \
                    .groupby("production_companies")["overview"] \
                    .apply(lambda titles:\
                               np.mean(titles.str.split(' ').apply(lambda it: len(it))) ) \
                    .sort_values(ascending=False)

answer_25 = company_overview.index[0]
print(f"Answer 25: '{answer_25}' ({company_overview.iloc[0]})")

Answer 25: 'Midnight Picture Show' (175.0)


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [65]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave' # +

In [66]:
# Последний процентиль из 100 (0.99 или 99ᵗʰ)
best_rating = data[data["vote_average"] >= data["vote_average"].quantile(0.99)] \
                .sort_values(by="vote_average", ascending=False)

# ===========
# Можно уменьшить число строк в итоговой выборке так, чтобы
# число лучших фильмом по отношению к их полному числу
# именно в _этом конкретном_ входном наборе данных
# составляло действительно число, близкое к 1% (_проценту_).
# Если раскомментировать код ниже, то получается число фильмов ~= 0.95% от общего.
# При этом ответ вычисляется такой же.

# percent = 1.0
# total_film_number = len(data)
# film_percentage = lambda: ( len(best_rating) / total_film_number) * 100

# films_found = film_percentage()
# if film_percentage() > percent:
#     print(f"Percent of best films if greater than {percent}%: {round(film_percentage(), 2)}%; droping rows...")
#     while film_percentage() > percent:
#         best_rating.drop(best_rating.tail(1).index, inplace=True)
#     print(f"Percent of best films is now {round(film_percentage(), 2)}%.")

# ===========

# ===========
# Поскольку один процент оставшихся лучших фильмов (по процентилю 0.99) всё равно представляет
# собой достаточно большое число в абсолютном выражении, а других
# более точных критериев поиска нет, то я всё же решил проверить итоговую таблицу
# по _всем_ вариантам ответа из теста, чтобы не искать вручную.
check_films_in = lambda films: \
                    len(best_rating[best_rating.original_title.isin(films)]) == len(films)

answer_options = [
    ["Inside Out", "The Dark Knight", "12 Years a Slave"],
    ["BloodRayne", "The Adventures of Rocky & Bullwinkle"],
    ["Batman Begins", "The Lord of the Rings: The Return of the King", "Upside Down"],
    ["300", "Lucky Number Slevin", "Kill Bill: Vol. 1"],
    ["Upside Down", "Inside Out", "Iron Man"]
]
# ===========

# ===========
# Получение ответа
answer_26 = next(answer_option for answer_option in answer_options if check_films_in(answer_option))
answer_26 = ", ".join(answer_26)
print(f"Answer 26: '{answer_26}'")
# ===========

# ===========
# Распечатка выборки лучших фильмов
best_rating

Answer 26: 'Inside Out, The Dark Knight, 12 Years a Slave'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,season
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",2008-07-16,8.1,2008,816921825,Summer
9,tt2096673,175000000,853708609,Inside Out,"[Amy Poehler, Phyllis Smith, Richard Kind, Bil...",Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,"[Comedy, Animation, Family]","[Walt Disney Pictures, Pixar Animation Studios...",2015-06-09,8.0,2015,678708609,Summer
34,tt3170832,6000000,35401758,Room,"[Brie Larson, Jacob Tremblay, Joan Allen, Sean...",Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,"[Drama, Thriller]","[Element Pictures, No Trace Camping, A24, Dupe...",2015-10-16,8.0,2015,29401758,Autumn
118,tt0816692,165000000,621752480,Interstellar,"[Matthew McConaughey, Jessica Chastain, Anne H...",Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,"[Adventure, Drama, Science Fiction]","[Paramount Pictures, Legendary Pictures, Warne...",2014-11-05,8.0,2014,456752480,Autumn
125,tt2084970,14000000,233555708,The Imitation Game,"[Benedict Cumberbatch, Keira Knightley, Matthe...",Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,"[History, Drama, Thriller, War]","[Black Bear Pictures, Bristol Automotive]",2014-11-14,8.0,2014,219555708,Autumn
370,tt1375666,160000000,825500000,Inception,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",Christopher Nolan,Your mind is the scene of the crime.,"Cobb, a skilled thief who commits corporate es...",148,"[Action, Thriller, Science Fiction, Mystery, A...","[Legendary Pictures, Warner Bros., Syncopy]",2010-07-14,7.9,2010,665500000,Summer
1191,tt2024544,20000000,187000000,12 Years a Slave,"[Chiwetel Ejiofor, Michael Fassbender, Lupita ...",Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,"[Drama, History]","[Plan B Entertainment, Regency Enterprises, Ri...",2013-10-18,7.9,2013,167000000,Autumn
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,"[Leonardo DiCaprio, Jonah Hill, Margot Robbie,...",Martin Scorsese,EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,"[Crime, Drama, Comedy]","[Paramount Pictures, Appian Way, EMJAG Product...",2013-12-25,7.9,2013,292000694,Winter
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,"[Elijah Wood, Ian McKellen, Viggo Mortensen, L...",Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,"[Adventure, Fantasy, Action]","[WingNut Films, New Line Cinema]",2003-12-01,7.9,2003,1024888979,Winter
872,tt0253474,35000000,120072577,The Pianist,"[Adrien Brody, Thomas Kretschmann, Frank Finla...",Roman Polanski,Music was his passion. Survival was his master...,The Pianist is a film adapted from the biograp...,150,"[Drama, War]","[Bac Films, Canal+Polska, Heritage Films, Stud...",2002-09-24,7.9,2002,85072577,Autumn


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [67]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint' # +

In [68]:
participations = pd.pivot_table(data.explode("cast"),
                                index=["imdb_id"], columns=["cast"], values=["original_title"],
                                aggfunc=lambda x: len(x) != 0, fill_value=False)

participations = participations["original_title"]
print(f"Row initial count: {len(participations)}")

participations = participations[participations.sum(axis=1) > 1]
print(f"Removed single actor films: {len(participations)}")

actors = participations.columns.tolist()
actor_colleage_counter = Counter()
actor_eq_colleague = False
for actor in actors:
    actor_films = participations[participations[actor] == True]

    # Выбираем колонки актёров, включая текущего actor,
    # у которых количество значений True (= 1) в из колонках в сумме больше одного.
    actor_films = actor_films.loc[:, actor_films.sum(axis=0) > 1]
    if len(actor_films.columns) == 0 or len(actor_films.columns) == 1:
        # Актёр мог играть во многих фильмах (его колонка заполнена одними True),
        # но лишь по одному разу с каждым из актёров в этих фильмах
        continue

    # Добавляем строку с итоговыми суммами всех True (= 1) в колонке каждого актёра,
    # с которыми actor играл совместно; там будут числа >= 2
    actor_films = actor_films.append(actor_films.sum(), ignore_index=True)

    # 1. Берём список колонок в подтаблице actor_films для актёра actor,
    # игравшего с другими актёрами
    actor_films_columns = actor_films.columns.tolist()
    # 2. Выясняем индекс колонки актёра actor, чтобы исключить из подсчёта
    # все колонки актёров до колонки actor (мы уже по ним прошлись на предыдущих итерациях,
    # составив уже пары актёров, включая текущего actor, как одного из коллег) и
    # саму колонку actor (чтобы не считать разы, когда актёр играл сам с собой).
    actor_index = actor_films_columns.index(actor)
    # 3. Берём все колонки актёров после колонки actor.
    sub_columns = actor_films_columns[actor_index + 1:] if actor_index < len(actor_films_columns) - 1 \
                                                        else []

    # Последняя строка таблицы, с итоговыми суммами по совместным фильмам,
    # преобразуется в объект Series, где индексом являются имена коллег актёра actor,
    # сам actor сюда не входит, поскольку подсчитываются его совместные работы с другими.
    actor_colleague_counts = actor_films.loc[len(actor_films)-1, sub_columns]
    for colleague, count in actor_colleague_counts.iteritems():
        # Обновляем счётчик actor_colleage_counter с ключом в виде
        # кортежа (актёр, его_коллега) и числом совместных работ (фильмов).
        actor_colleage_counter.update({(actor, colleague): count})

# Печатаем первые 15 пар актёров, которые чаще всего
# играли вместе в тех или иных фильмах.
answer_27 = list(map(lambda pair_count: (f"{pair_count[0][0]} & {pair_count[0][1]}", int(pair_count[1])),
                     actor_colleage_counter.most_common(15)))
answer_2_string = "\n".join([f"\t\u2714 {it[0]} -> {it[1]}" for it in answer_27])
print(f"Answer 27:\n{answer_2_string}")

Row initial count: 1889
Removed single actor films: 1888
Answer 27:
	✔ Daniel Radcliffe & Emma Watson -> 8
	✔ Daniel Radcliffe & Rupert Grint -> 8
	✔ Emma Watson & Rupert Grint -> 8
	✔ Ben Stiller & Owen Wilson -> 6
	✔ Helena Bonham Carter & Johnny Depp -> 6
	✔ Adam Sandler & Kevin James -> 5
	✔ Hugh Jackman & Ian McKellen -> 5
	✔ Kristen Stewart & Robert Pattinson -> 5
	✔ Kristen Stewart & Taylor Lautner -> 5
	✔ Paul Walker & Vin Diesel -> 5
	✔ Robert Pattinson & Taylor Lautner -> 5
	✔ Betsy Russell & Costas Mandylor -> 4
	✔ Betsy Russell & Tobin Bell -> 4
	✔ Brad Pitt & George Clooney -> 4
	✔ Brad Pitt & Julia Roberts -> 4


# Submission

In [69]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [70]:
# и убедиться что ни чего не пропустил)
len(answers)

27